In [3]:
import bionumpy as bnp
fastq_file = "C:/Users/admin/Desktop/SRR19127870.fastq"

# Opens the FASTQ file
f = bnp.open(fastq_file)

# Reads a chunk from the FASTQ file
chunk = f.read_chunk()

# Prints the chunk to verify its contents
print(chunk)


SequenceEntryWithQuality with 12606 entries
                     name                 sequence                  quality
  SRR19127870.1.1 M04500:  AAAAGGAAAAAAGAAACACGGAC  [30 30 30 30 30 30 30 3
  SRR19127870.1.2 M04500:  ACCCACACTCCAGTGGGCAGTCT  [30 30 30 30 30 30 30 3
  SRR19127870.2.1 M04500:  CTCTATGCAACAGAAGCAAAGAG  [30 30 30 30 30 30 30 3
  SRR19127870.2.2 M04500:  CTCCCGTGCAAGAGTAAGCATAC  [30 30 30 30 30 30 30 3
  SRR19127870.3.1 M04500:  ACCAAGTTGAAGGAATGCATGGA  [30 30 30 30 30 30 30 3
  SRR19127870.3.2 M04500:  GACCTAAGCTCATCCTTCACATA  [30 30 30 30 30 30 30 3
  SRR19127870.4.1 M04500:  GGTTTTGCGGGTCCGCCATGGCT  [30 30 30 30 30 30 30 3
  SRR19127870.4.2 M04500:  CTAACATCGCGTCTTTTCTATTC  [30 30 30 30 30 30 30 3
  SRR19127870.5.1 M04500:  GTCTGTCGTAACGGGCAACTCTG  [30 30 30 30 30 30 30 3
  SRR19127870.5.2 M04500:  CAATAGCTATATGGTAACAATCT  [30 30 30 30 30 30 30 3


In [7]:

import bionumpy as bnp
import numpy as np
fastq_file = "C:/Users/admin/Desktop/SRR19127870.fastq"

# Opens the FASTQ file
f = bnp.open(fastq_file)

# Reads a chunk from the FASTQ file
chunk = f.read_chunk()

# Prints the number of 'G' nucleotides in the sequences
num_Gs = np.sum(chunk.sequence == chunk.sequence.encoding.encode("G"))
print("Number of Gs in the sequences:")
print(num_Gs)

# Prints the number of sequences with mean quality > 30
mean_qualities = np.mean(chunk.quality, axis=1)
num_high_quality_sequences = np.sum(mean_qualities > 30)
print("Number of sequences with mean quality > 30:")
print(num_high_quality_sequences)



Number of Gs in the sequences:
369931
Number of sequences with mean quality > 30:
0


In [8]:
import bionumpy as bnp

def test(file="C:/Users/admin/Desktop/SRR19127870.fastq", out_filename="C:/Users/admin/Desktop/SRR19127870_filtered.fastq"):
    with bnp.open(out_filename, 'w') as out_file:
        for reads in bnp.open(file).read_chunks():
            min_quality_mask = reads.quality.min(axis=-1) > 1
            max_quality_mask = reads.quality.mean(axis=-1) > 10
            mask = min_quality_mask & max_quality_mask
            print(f'Filtering reads: {len(reads)} -> {mask.sum()}')
            out_file.write(reads[mask])

if __name__ == "__main__":
    test()


Filtering reads: 12606 -> 12421
Filtering reads: 12529 -> 12321
Filtering reads: 12260 -> 12036
Filtering reads: 8802 -> 8202


In [9]:
import bionumpy as bnp
from npstructures import ragged_slice

def test_bamquality():
    # Opens the alignments file
    alignments = bnp.open("C:/Users/admin/Downloads/alignments.bam").read()
    
    # Extracts the first cigar operation for each alignment
    start_cigar = alignments.cigar_op[..., 0]
    
    # Gets alignments that start with soft-clip
    start_clipped_alignments = alignments[start_cigar == "S"]
    
    # Gets the number of softclipped bases
    n_clipped_bases = start_clipped_alignments.cigar_length[..., 0]
    
    # Extracts clipped bases
    clipped_bases = ragged_slice(start_clipped_alignments.sequence, ends=n_clipped_bases)
    
    # Counts bases in softclipped regions
    print("Base counts in softclipped regions:")
    print(bnp.count_encoded(clipped_bases.ravel()))
    
    # Counts bases in whole reads
    print("Base counts in whole reads:")
    print(bnp.count_encoded(alignments.sequence.ravel()))

if __name__ == "__main__":
    test_bamquality()


Base counts in softclipped regions:
=: 0
A: 0
C: 0
M: 0
G: 0
R: 0
S: 0
V: 0
T: 0
W: 0
Y: 0
H: 0
K: 0
D: 0
B: 0
N: 0
Base counts in whole reads:
=: 0
A: 19
C: 8
M: 0
G: 10
R: 0
S: 0
V: 0
T: 12
W: 0
Y: 0
H: 0
K: 0
D: 0
B: 0
N: 0
